# Labo 5 Modèle word2vec et ses applications

## 1. Tester et évaluer un modèle déjà entraîné sur Google News

Veuillez télécharger le modèle word2vec pré-entraîné sur le corpus Google News en écrivant :
```python
from gensim import downloader as api
w2v_vectors = api.load("word2vec-google-news-300")
```
ce qui téléchargera le fichier la première fois.
Après avoir téléchargé le modèle, vous pourrez l’utiliser ainsi (dans le dossier gensim-data) :
```python
from gensim.models import KeyedVectors
w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)
```

In [1]:
# Takes ~30min to download. Only execute once
from gensim import downloader as api
w2v_vectors = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [2]:
# Model is then located at home directory, adjust as needed.
import os
file_path = os.path.join(os.path.expanduser("~"), "gensim-data", "word2vec-google-news-300", "word2vec-google-news-300.gz")

In [3]:
# Takes ~1min
from gensim.models import KeyedVectors
w2v_vectors = KeyedVectors.load_word2vec_format(file_path, binary=True)

### a. Quelle place en mémoire occupe le processus du notebook avec les vecteurs de mots ?

In [4]:
import psutil
import os

# Get the current process ID
process = psutil.Process(os.getpid())

# Get memory information in bytes and convert to MB
memory_info = process.memory_info()
memory_usage_mb = memory_info.rss / 1024 / 1024  # Convert to MB

print(f"Memory usage of the notebook process: {memory_usage_mb:.2f} MB")
print(f"Word2vec model size: {w2v_vectors.vectors.nbytes / 1024 / 1024:.2f} MB")

Memory usage of the notebook process: 1024.30 MB
Word2vec model size: 3433.23 MB


### b. Quelle est la dimension de l’espace vectoriel dans lequel les mots sont représentés ?

In [5]:
print(f"Vector dimension: {w2v_vectors.vector_size}")

Vector dimension: 300


### c. Quelle est la taille du vocabulaire connu du modèle ? Veuillez afficher cinq mots anglais qui sont dans le vocabulaire et deux qui ne le sont pas.


In [6]:
# Get vocabulary size
vocab_size = len(w2v_vectors)
print(f"Vocabulary size: {vocab_size}")

# Five words in vocabulary
words_in_vocab = ['baste', 'spleen', 'maudlin', 'saudade', 'aa']
print("\nFive words in vocabulary:")
for word in words_in_vocab:
    if word in w2v_vectors:
        print(f"'{word}' is in vocabulary")
    else:
        print(f"'{word}' is NOT in vocabulary")

# Two words not in vocabulary
words_not_in_vocab = ['enneahedron', 'petrichor']
print("\nTwo words not in vocabulary:")
for word in words_not_in_vocab:
    if word in w2v_vectors:
        print(f"'{word}' is in vocabulary")
    else:
        print(f"'{word}' is NOT in vocabulary")

Vocabulary size: 3000000

Five words in vocabulary:
'baste' is in vocabulary
'spleen' is in vocabulary
'maudlin' is in vocabulary
'saudade' is in vocabulary
'aa' is in vocabulary

Two words not in vocabulary:
'enneahedron' is NOT in vocabulary
'petrichor' is NOT in vocabulary


### d. Quelle est la similarité entre les mots rabbit et carrot ? Veuillez rappeler comment on mesure les similarités entre deux mots grâce à leurs vecteurs.

In [7]:
similarity = w2v_vectors.similarity('rabbit', 'carrot')
print(f"Similarity between 'rabbit' and 'carrot': {similarity:.4f}")

Similarity between 'rabbit' and 'carrot': 0.3631


> TODO answer

### e. Considérez au moins 5 paires de mots anglais, certains proches par leurs sens, d’autres plus éloignés. Pour chaque paire, calculez la similarité entre les deux mots. Veuillez indiquer si les similarités obtenues correspondent à vos intuitions sur la proximité des sens des mots.

In [8]:
pairs = [
    ('king', 'queen'),
    ('bad', 'bunny'),
    ('good', 'bunny'),
    ('plane', 'tower'),
    ('duc', 'duchesse')
]

for pair in pairs:
    word1, word2 = pair
    if word1 in w2v_vectors and word2 in w2v_vectors:
        similarity = w2v_vectors.similarity(word1, word2)
        print(f"Similarity between '{word1}' and '{word2}': {similarity:.4f}")
    else:
        print(f"One or both words in the pair '{pair}' are not in the vocabulary.")

Similarity between 'king' and 'queen': 0.6511
Similarity between 'bad' and 'bunny': 0.1326
Similarity between 'good' and 'bunny': 0.1150
Similarity between 'plane' and 'tower': 0.2549
Similarity between 'duc' and 'duchesse': 0.3245


> TODO answer and/or pick better words

### f. Pouvez-vous trouver des mots de sens opposés mais qui sont proches selon le modèle ? Comment expliquez-vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?

In [9]:
similarity = w2v_vectors.similarity('black', 'white')
print(f"Similarity: {similarity:.4f}")

Similarity: 0.8092


>TODO answer

### g. En vous aidant de la documentation de Gensim sur KeyedVectors, obtenez les scores du modèle word2vec sur les données de test WordSimilarity-353. Veuillez rappeler en 1-2 phrases comment les différents scores sont calculés.

In [10]:
from gensim.test.utils import datapath

word_sim_353_path = datapath('wordsim353.tsv')
result = w2v_vectors.evaluate_word_pairs(word_sim_353_path)

print(f"Pearson correlation coefficient: {result[0][0]:.4f}")
print(f"Pearson correlation p-value: {result[0][1]:.4f}")
print(f"Spearman rank correlation coefficient: {result[1][0]:.4f}")
print(f"Spearman rank correlation p-value: {result[1][1]:.4f}")
print(f"Ratio of pairs with OOV words: {result[2]:.2%}")

Pearson correlation coefficient: 0.6239
Pearson correlation p-value: 0.0000
Spearman rank correlation coefficient: 0.6589
Spearman rank correlation p-value: 0.0000
Ratio of pairs with OOV words: 0.00%


> TODO answer

### h. En vous aidant de la documentation, calculez le score du modèle word2vec sur les données questions-words.txt. Attention, cette évaluation prend une dizaine de minutes, donc il vaut mieux commencer par tester avec un fragment de ce fichier (copier/coller les 100 premières analogies). Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [11]:
from gensim.test.utils import datapath
import time

analogies_path = datapath('questions-words.txt')

start_time = time.time()

result = w2v_vectors.evaluate_word_analogies(analogies_path)

elapsed_time = time.time() - start_time

print(f"Total accuracy: {result[0]:.4f}")
print(f"Number of sections: {len(result[1])}")
print(f"Evaluation time: {elapsed_time:.2f} seconds")

print("\nSection-wise results:")
for section_name, section_result in result[1]:
    print(f"Section '{section_name}': {section_result[0]:.4f} accuracy ({section_result[1]} correct out of {section_result[2]} total)")

Total accuracy: 0.7401
Number of sections: 15
Evaluation time: 170.53 seconds

Section-wise results:


ValueError: too many values to unpack (expected 2)

## 2. Entraîner deux nouveaux modèles word2vec à partir de deux corpus

### a. En utilisant gensim.downloader (voir question 1) récupérez le corpus qui contient les 108 premiers caractères de Wikipédia (en anglais) avec la commande : corpus = api.load('text8'). Combien de phrases et de mots (tokens) possède ce corpus ?

In [12]:
corpus = api.load('text8')
sentences = list(corpus)

num_sentences = len(sentences)
num_tokens = sum(len(sentence) for sentence in sentences)

print(f"Nombre de phrases : {num_sentences}")
print(f"Nombre de mots (tokens) : {num_tokens}")

[==================================================] 100.0% 31.6/31.6MB downloaded
Nombre de phrases : 1701
Nombre de mots (tokens) : 17005207


### b. Entraînez un nouveau modèle word2vec sur ce nouveau corpus (voir la documentation de Word2vec). Si nécessaire, procédez progressivement, en commençant par utiliser 1% du corpus, puis 10%, etc., pour contrôler le temps nécessaire.
- Veuillez indiquer la dimension choisie pour le embedding de ce nouveau modèle.
- Combien de temps prend l’entraînement sur le corpus total ?
- Quelle est la taille (en Mo) du modèle word2vec résultant ?

In [14]:
from gensim.models import Word2Vec

In [15]:
embedding_dim = 100

corpus_sizes = [0.01, 0.1, 1.0]
for size in corpus_sizes:
    print(f"\nEntraînement avec {int(size * 100)}% du corpus :")
    subset = sentences[:int(len(sentences) * size)]

    start_time = time.time()
    model = Word2Vec(sentences=subset, vector_size=embedding_dim, window=5, min_count=5, sg=1, epochs=10)
    training_time = time.time() - start_time
    
    model_file = f"text8_word2vec_{int(size * 100)}.model"
    model.save(model_file)
    
    model_size_mb = os.path.getsize(model_file) / (1024 * 1024)
    
    print(f"Temps d'entraînement : {training_time:.2f} secondes")
    print(f"Taille du modèle : {model_size_mb:.2f} Mo")


Entraînement avec 1% du corpus :
Temps d'entraînement : 1.57 secondes
Taille du modèle : 3.18 Mo

Entraînement avec 10% du corpus :
Temps d'entraînement : 18.05 secondes
Taille du modèle : 15.41 Mo

Entraînement avec 100% du corpus :
Temps d'entraînement : 221.69 secondes
Taille du modèle : 56.47 Mo


### c. Mesurez la qualité de ce modèle comme en (1g) et (1h). Ce modèle est-il meilleur que celui entraîné sur Google News ? Quelle est selon vous la raison de la différence ?

### d. Téléchargez maintenant le corpus quatre fois plus grand constitué de la concaténation du corpus text8 et des dépêches économiques de Reuters fourni par l’enseignant et appelé wikipedia_augmented.zip (à décompresser en un fichier ‘.dat’ de 413 Mo). Entraînez un nouveau modèle word2vec sur ce corpus, en précisant la dimension choisie pour les embeddings.
- Utilisez la classe Text8Corpus() pour charger ce corpus, ce qui fera automatiquement la
tokenisation et la segmentation en phrases.
- Combien de temps prend l’entraînement ?
- Quelle est la taille (en Mo) du modèle word2vec résultant ?

In [16]:
from gensim.models.word2vec import Text8Corpus

In [20]:
corpus_path = "wikipedia_augmented.dat"
corpus = Text8Corpus(corpus_path)

embedding_dim = 100

start_time = time.time()
model = Word2Vec(sentences=corpus, vector_size=embedding_dim, window=5, min_count=5, sg=1, epochs=10)
training_time = time.time() - start_time

model_file = "wikipedia_augmented_word2vec.model"
model.save(model_file)

model_size_mb = os.path.getsize(model_file) / (1024 * 1024)

print(f"Temps d'entraînement : {training_time:.2f} secondes")
print(f"Taille du modèle : {model_size_mb:.2f} Mo")

Temps d'entraînement : 1054.53 secondes
Taille du modèle : 3.71 Mo


### e. Testez ce modèle comme en (1g) et (1h). Est-il meilleur que le précédent ?